<a href="https://colab.research.google.com/github/KangTuna/Machine-learning-and-deep-learning-practices/blob/main/RNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
# 디바이스 할당
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/RNN/Early_detection_data.xlsx')

In [ ]:
first_line = 0.2
second_line = 0.1
slop_line = 1.5

In [ ]:
# Label 정의
for i in range(2,len(data)):
  if data.loc[i-2,'Rate'] == 0:
    temp_slop = 15*data.loc[i,'Rate']
  else:
    temp_slop = data.loc[i,'Rate']/data.loc[i-2,'Rate']

  if data.loc[i-2,'Rate'].mean()>first_line:
    data.loc[i,'label'] = 1
  elif data.loc[i,'Rate']>second_line and temp_slop>slop_line:
    data.loc[i,'label'] = 1
  else:
    data.loc[i,'label'] = 0

In [ ]:
features = ['Week','Average temperature', 'Minimum temperature','Maximum temperature',
            'Rainfall','Minimum humidity','Relative humidity','Day length',
            'Duration of sunshine','Soil temperature at 3m','Soil temperature at 5m']

In [ ]:
start_year = 2009
pred_year = 2015
window_size = 3

In [ ]:
train = data[:53*(pred_year - start_year)]
test = data[53*(pred_year - start_year) - window_size:]

In [ ]:
train_x = train.loc[:,features]
train_label = train[['label']]

In [ ]:
test_x = test.loc[:,features]
test_label = test[['label']]

In [ ]:
X_max = train_x.max()
X_min = train_x.min()
X_max_test = test_x.max()
X_min_test = test_x.min()

In [ ]:
train_x = (train_x - X_min) / (X_max - X_min)
test_x = (test_x - X_min_test) / (X_max_test - X_min_test)

In [ ]:
def make_dataset(x_data, label, window_size):
  x_list = []
  label_list = []
  for i in range(len(x_data) - window_size):
    x_list.append(np.array(x_data.iloc[i:i+window_size]))
    label_list.append(np.array(label.iloc[i+window_size]))
  x_list = torch.Tensor(np.array(x_list))
  label_list = torch.Tensor(np.array(label_list)).view(-1,1)
  return x_list, label_list

In [ ]:
x_data,label_data = make_dataset(train_x, train_label,window_size)
x_test,label_test = make_dataset(test_x, test_label,window_size)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder()

In [ ]:
one_hot_encoded_train = encoder.fit_transform(label_data)
one_hot_encoded_test = encoder.fit_transform(label_test)

In [ ]:
label_data = torch.Tensor(one_hot_encoded_train.toarray())
label_test = torch.Tensor(one_hot_encoded_test.toarray())

In [ ]:
x_data = x_data.to(device)
label_data = label_data.to(device)

x_test = x_test.to(device)
label_test = label_test.to(device)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
train_dataset = TensorDataset(x_data,label_data)

In [ ]:
batch_size = 30

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = batch_size)

In [ ]:
hidden_size = 5

In [ ]:
class RNN(nn.Module):
  def __init__(self):
    super(RNN, self).__init__()

    self.rnn = nn.RNN(input_size = len(features), hidden_size = hidden_size, num_layers = 5,
                      batch_first = True)
    
    self.fc1 = nn.Linear(in_features = window_size*hidden_size, out_features = 10)
    self.fc2 = nn.Linear(in_features = 10, out_features = 2)

    self.relu = nn.ReLU()

  def forward(self,x,h0):
    x, hn = self.rnn(x,h0)

    x = x.reshape(x.size(0),-1)

    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)

    #x = x.reshape(-1)

    return x


In [ ]:
#난수 고정
torch.cuda.manual_seed(123)

model = RNN()
model.to(device)
lr = 0.0002
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 손실 함수 : 교차 엔트로피 함수
criterion = nn.CrossEntropyLoss()

loss_train_record = []
loss_test_record = []

In [ ]:
num_epochs= 500

early_stopping_patience = 20
epochs_without_imporvement = 0
stop_training = False

best_val_loss = float('inf')
print(best_val_loss)
for epoch in range(num_epochs):
  if stop_training:
    break
  for data,label in train_loader:

    inputs = data.to(device)
    labels = label.to(device)
    optimizer.zero_grad()
    h0 = torch.zeros(5, inputs.shape[0], hidden_size).to(device)
    outputs = model(inputs,h0)
    loss_train = criterion(outputs, labels)

    loss_train.backward()

    optimizer.step()
  loss_train_record.append(loss_train.item())

  h_test = torch.zeros(5, x_test.shape[0], hidden_size).to(device)
  outputs_test = model(x_test,h_test)
  loss_test = criterion(outputs_test, label_test)
  loss_test_record.append(loss_test.item())
  # print(f'best_val_loss {best_val_loss} loss_test {loss_test}')
  if loss_test < best_val_loss:
    best_val_loss = loss_test
    epochs_without_imporvement = 0
  else:
    epochs_without_imporvement += 1
    if epochs_without_imporvement >= early_stopping_patience:
      stop_training = True
  
  if epoch % 100 == 0:
    print(f'epoch {epoch} loss : {loss_train.item()}')

inf
epoch 0 loss : 0.14793749153614044


In [ ]:
|